# Laboratorio 1: Aprendizaje No Supervisado
Reglas de asociación entre películas

- Juan Pablo Analian

Obtener reglas de asociación entre películas en el dataset movielens como si fuera recomendación.
Aplicar diferentes métricas de ordenamiento. Hacer un pequeño informe

In [1]:
import numpy as np
import pandas as pd
from itertools import combinations, groupby
from collections import Counter
from efficient_apriori import apriori

Limitamos la cantidad de registros a tomar del CSV Ratings, para evitar problemas de memoria en el equipo donde hacemos pruebas.

In [2]:
df_movies = pd.read_csv('./movies.csv')
df_rat = pd.read_csv('./ratings.csv' ,nrows=5000000) 

In [3]:
print('movies', len(df_movies))
display(df_movies.head(10))
print('ratings', len(df_rat))
display(df_rat.head(10))

movies 27278


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


ratings 5000000


,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580
5,1,112,3.5,1094785740
6,1,151,4.0,1094785734
7,1,223,4.0,1112485573
8,1,253,4.0,1112484940
9,1,260,4.0,1112484826


In [4]:
df_rat.rating.value_counts().sort_index(ascending=False)

5.0     719276
4.5     381307
4.0    1383159
3.5     553094
3.0    1075743
2.5     225091
2.0     361178
1.5      69933
1.0     170911
0.5      60308
Name: rating, dtype: int64

Vamos a tomar solo las peliculas cuyo rating fue de 4 o superiro. Entendiendo que si alguien le da esta calificación, es una película candidata a ser recomendada.

In [5]:
print("Cantidad de registros con rating >= a 4:",len(df_rat[df_rat.rating >= 4.0]))
print("Como se distribuyen estas cantidades, segun el rating")
df_rat[df_rat.rating >= 4.0].rating.value_counts()

Cantidad de registros con rating >= a 4: 2483742
Como se distribuyen estas cantidades, segun el rating


4.0    1383159
5.0     719276
4.5     381307
Name: rating, dtype: int64

In [6]:
df_rat.loc[df_rat.rating >= 4.0, ['userId','movieId']].head(5)

,userId,movieId
6,1,151
7,1,223
8,1,253
9,1,260
10,1,293


In [7]:
df_tran = pd.merge(df_rat.loc[df_rat.rating >= 4.0, ['userId','movieId']], df_movies[['movieId','title']] ,on='movieId', how= "inner") 
df_tran = df_tran.sort_values(by='userId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
display(df_tran.head(10))

,userId,movieId,title
0,1,151,Rob Roy (1995)
127235,1,1333,"Birds, The (1963)"
128897,1,1358,Sling Blade (1996)
131346,1,1374,Star Trek II: The Wrath of Khan (1982)
133706,1,1387,Jaws (1975)
137098,1,1967,Labyrinth (1986)
138196,1,2021,Dune (1984)
139145,1,2100,Splash (1984)
140137,1,2118,"Dead Zone, The (1983)"
126273,1,1321,"American Werewolf in London, An (1981)"


In [8]:
df_tran2 = df_tran.values[:,[0,2]]
print(df_tran2[0:10])

[[1 'Rob Roy (1995)']
 [1 'Birds, The (1963)']
 [1 'Sling Blade (1996)']
 [1 'Star Trek II: The Wrath of Khan (1982)']
 [1 'Jaws (1975)']
 [1 'Labyrinth (1986)']
 [1 'Dune (1984)']
 [1 'Splash (1984)']
 [1 'Dead Zone, The (1983)']
 [1 'American Werewolf in London, An (1981)']]


In [9]:
transactions=[]
for user_id, movie in groupby(df_tran2, lambda x: x[0]):
    transactions.append([item[1] for item in movie])

print("La cantidad de películas vistas por el userID 1 es de", len(transactions[0]))

La cantidad de películas vistas por el userID 1 es de 88


In [10]:
itemsets, rules = apriori(transactions, min_support=0.15,  min_confidence=0.5)

In [11]:
print("itemset len:",len(itemsets))
print("rules len:",len(rules))

itemset len: 3
rules len: 59


In [12]:
print(rules[0])

{Braveheart (1995)} -> {Forrest Gump (1994)} (conf: 0.596, supp: 0.166, lift: 1.733, conv: 1.625)


In [13]:
print(rules[0].lhs, type(rules[0].lhs))
print(rules[0].rhs)
print(rules[0].lift)
print(rules[0].support) 
print(rules[0].confidence) 
print(rules[0].num_transactions)

('Braveheart (1995)',) <class 'tuple'>
('Forrest Gump (1994)',)
1.7334371655331744
0.16569115634372908
0.5961860854987426
34341


In [14]:
columns = ['lhs', 'lhs_count', 'rhs','rhs_count', 'support', 'lift', 'confidence']
list_of_rules=[]
for r in rules:
    list_of_rules.append([r.lhs, len(r.lhs), r.rhs, len(r.rhs), r.support,r.lift, r.confidence])

df_res = pd.DataFrame(list_of_rules, columns=columns)


### Exploramos los datos ordenando por *support*, *confidence* y *lift*
Solo para ver y entender las recomendaciones que se harían.

In [15]:
df_res.sort_values(by='support', ascending=False)

,lhs,lhs_count,rhs,rhs_count,support,lift,confidence
25,"(Pulp Fiction (1994),)",1,"(Shawshank Redemption, The (1994),)",1,0.232899,1.528643,0.618561
24,"(Shawshank Redemption, The (1994),)",1,"(Pulp Fiction (1994),)",1,0.232899,1.528643,0.575561
27,"(Pulp Fiction (1994),)",1,"(Silence of the Lambs, The (1991),)",1,0.227454,1.674769,0.604099
26,"(Silence of the Lambs, The (1991),)",1,"(Pulp Fiction (1994),)",1,0.227454,1.674769,0.630580
39,"(Silence of the Lambs, The (1991),)",1,"(Shawshank Redemption, The (1994),)",1,0.226493,1.551761,0.627916
40,"(Shawshank Redemption, The (1994),)",1,"(Silence of the Lambs, The (1991),)",1,0.226493,1.551761,0.559729
45,"(Star Wars: Episode IV - A New Hope (1977),)",1,(Star Wars: Episode V - The Empire Strikes Bac...,1,0.211671,2.620548,0.679155
44,(Star Wars: Episode V - The Empire Strikes Bac...,1,"(Star Wars: Episode IV - A New Hope (1977),)",1,0.211671,2.620548,0.816742
8,"(Shawshank Redemption, The (1994),)",1,"(Forrest Gump (1994),)",1,0.207187,1.488714,0.512018
9,"(Forrest Gump (1994),)",1,"(Shawshank Redemption, The (1994),)",1,0.207187,1.488714,0.602405


In [16]:
df_res.sort_values(by='confidence', ascending=False)

,lhs,lhs_count,rhs,rhs_count,support,lift,confidence
53,(Star Wars: Episode V - The Empire Strikes Bac...,2,"(Star Wars: Episode IV - A New Hope (1977),)",1,0.161061,2.858179,0.890804
14,"(Lord of the Rings: The Two Towers, The (2002),)",1,(Lord of the Rings: The Fellowship of the Ring...,1,0.157421,4.131884,0.855109
54,"(Star Wars: Episode IV - A New Hope (1977), St...",2,(Star Wars: Episode V - The Empire Strikes Bac...,1,0.161061,3.161247,0.819286
44,(Star Wars: Episode V - The Empire Strikes Bac...,1,"(Star Wars: Episode IV - A New Hope (1977),)",1,0.211671,2.620548,0.816742
46,(Star Wars: Episode VI - Return of the Jedi (1...,1,"(Star Wars: Episode IV - A New Hope (1977),)",1,0.196587,2.589462,0.807053
55,"(Star Wars: Episode IV - A New Hope (1977), St...",2,(Star Wars: Episode VI - Return of the Jedi (1...,1,0.161061,3.123748,0.760902
15,(Lord of the Rings: The Fellowship of the Ring...,1,"(Lord of the Rings: The Two Towers, The (2002),)",1,0.157421,4.131884,0.760659
48,(Star Wars: Episode VI - Return of the Jedi (1...,1,(Star Wars: Episode V - The Empire Strikes Bac...,1,0.180804,2.864037,0.742259
23,"(Seven (a.k.a. Se7en) (1995),)",1,"(Pulp Fiction (1994),)",1,0.169506,1.929858,0.726626
50,"(Shawshank Redemption, The (1994), Silence of ...",2,"(Pulp Fiction (1994),)",1,0.158935,1.863717,0.701723


In [17]:
df_res.sort_values(by='lift', ascending=False)

,lhs,lhs_count,rhs,rhs_count,support,lift,confidence
15,(Lord of the Rings: The Fellowship of the Ring...,1,"(Lord of the Rings: The Two Towers, The (2002),)",1,0.157421,4.131884,0.760659
14,"(Lord of the Rings: The Two Towers, The (2002),)",1,(Lord of the Rings: The Fellowship of the Ring...,1,0.157421,4.131884,0.855109
57,(Star Wars: Episode V - The Empire Strikes Bac...,1,"(Star Wars: Episode IV - A New Hope (1977), St...",2,0.161061,3.161247,0.621461
54,"(Star Wars: Episode IV - A New Hope (1977), St...",2,(Star Wars: Episode V - The Empire Strikes Bac...,1,0.161061,3.161247,0.819286
56,(Star Wars: Episode VI - Return of the Jedi (1...,1,"(Star Wars: Episode IV - A New Hope (1977), St...",2,0.161061,3.123748,0.661207
55,"(Star Wars: Episode IV - A New Hope (1977), St...",2,(Star Wars: Episode VI - Return of the Jedi (1...,1,0.161061,3.123748,0.760902
49,(Star Wars: Episode V - The Empire Strikes Bac...,1,(Star Wars: Episode VI - Return of the Jedi (1...,1,0.180804,2.864037,0.697640
48,(Star Wars: Episode VI - Return of the Jedi (1...,1,(Star Wars: Episode V - The Empire Strikes Bac...,1,0.180804,2.864037,0.742259
58,"(Star Wars: Episode IV - A New Hope (1977),)",1,(Star Wars: Episode V - The Empire Strikes Bac...,2,0.161061,2.858179,0.516771
53,(Star Wars: Episode V - The Empire Strikes Bac...,2,"(Star Wars: Episode IV - A New Hope (1977),)",1,0.161061,2.858179,0.890804


## Resumen - Informe

En primer lugar, para evitar problemas de memoria y procesamiento de la maquina local, hemos tomado una muestra de los primeros 5 millones de registros (del total de 20) que tiene el dataset de ratings.

Para poder recomendar una película en base a otra, hemos filtrado de este dataset de 5 millones, aquellas películas cuya puntuación es mayor o igual a cuatro, obteniendo un total de 2.4 millones de registros. Entendemos que si una persona aplica este puntaje, es porque puede recomendarla.

Las transacciones fueron armadas en base a lo que ha visto (puntuado) cada usuario. Una transaccion, tiene todas las películas vistas por el usuario.

Aplicamos el algoritmo con estos valores:  min_support=0.15 y min_confidence=0.5. En donde especificamos que encuentre las películas que han sido vistas *por lo menos* por el 15% del total de los usuaros (ó transacciones) y que tenga una confianza de 50%, es decir que la probabilidad que alguien que haya visto la película A, tambien haya visto la B sea por lo menos el 50%.

El resultado con sus reglas lo pasamos a un dataset, simplemente para mostrar las reglas ordenadas según los valores asignados por el algoritmo apriori.

> Como resumen del dataset, podemos decir que:
  - Unas de las películas mas vista (de las que un usuario recomendaría) es: *Pulp Fiction* y *The Shawshank Redemption*. Esto es ordenando el dataset por la columna "support".
  - Es muy probable que quienes vieron "Star Wars x" sigan viendo otra pelicula de la misma saga. Lo mismo podríamos decir del "Señor de los anillos". Tomando el valor de "confidence".
  - Las películas mas relacionadas entre sí serían la de la serie del señor de los anillos.  ordenando el dataset por la columna "lift", que nos indica que la probabilidad de que estas películas se vean juntas es la mayor.

